In [27]:
import tensorflow as tf
import timeit

#### TensorFlow Graph 是什麼

在一般情況下，我們可以直接執行 TensorFlow 的 Operation，像是直接執行 Python Statement 那樣，此時我們稱為 Run TensorFlow Eagerly 或是 Eager Execution。

除了 Eager Execution 外，TensorFlow 還有另外一種 Execution 方式稱為 Graph Execution。Graph Execution 將所有的 tf.Operation 與 tf.Tensor 包在 tf.Graph 中。

tf.Graph 是一種 Data Structure，紀錄所有的 Computation (tf.Operaarion) 與 Data (tf.Tensor)，因此可以不必在 Python 的環境下執行，且比 Eager Execution 擁有更高的效能。

下圖為 2 個 Layer 的 Neural Network 的 Graph。

<img src="img/04/01.png" width="600"/>

#### TensorFlow Graph 的優點

- 可攜性高：可以在非 Python 環境下使用 Graph
- 速度快：TensorFlow runtime 使用 Grappler optimize TensorFlow Graph

#### 透過 tf.function() 建立一個 TensorFlow Function

tf.function() 可以將 Python 中一般的 Function 轉為 TensorFlow Function。TensorFlow Function 的呼叫方式與一般的 Python Function 一樣。當 TensorFlow Function 被呼叫時，會將其所包含的 Operation 轉為一個 TensorFlow Graph，再執行這一個 Graph

In [3]:
def regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

tensorflow_function = tf.function(regular_function)

In [4]:
x1 = tf.constant([
    [1.0, 2.0]
])

y1 = tf.constant([
    [2.0],
    [3.0]
])

b1 = tf.constant(4.0)

In [5]:
tensorflow_function(x1, y1, b1).numpy()

2021-12-28 08:58:31.522581: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-28 08:58:31.524385: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


array([[12.]], dtype=float32)

In [6]:
regular_function(x1, y1, b1).numpy()

array([[12.]], dtype=float32)

除了透過 tf.function() 建立 TensorFlow Function，也可以透過 Decorator 建立

In [7]:
@tf.function
def regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

In [8]:
regular_function

In [9]:
regular_function(x1, y1, b1).numpy()

array([[12.]], dtype=float32)

#### Convert Python logic to TensorFlow Graph

由上面的 Code 可以發現我們通常會在一般 Function 中寫 TensorFlow Operation 與 Python Logic (if, else, for, while, continue, break)。當我們將一般 Function 轉為 TensorFlow Function 後，再執行這一個 TensorFlow Function，TensorFlow Function 中所有的 Operation 會被建立成一個 TensorFlow Graph。換句話說，TensorFlow Function 中的所有 Code 會被轉為 graph-generating code。

TensorFlow Operation 可以比較簡單的被轉為 graph-generating code；Python logic 則需要透過 tf.autograph 將其轉為 graph-generating code。

In [10]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0
    
tf_simple_relu = tf.function(simple_relu)

In [11]:
# graph-generating code
print(tf.autograph.to_code(simple_relu))

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

#### Polymorphism: One Function, Many Graphs

已知 TensorFlow Function 會建立 TensorFlow Graph，因此傳入 TensorFlow Function 的參數可以視為 TensorFlow Graph 的 Input。Input 的 dtype 與 shape 會紀錄在 Input Signiture。每一個 TensorFlow Graph 都只會有對應的「一個」Input Signiture。

當我們傳入不同 dtype 或 shape 的參數到 TensorFlow Function 時，相對應的 TensorFlow Graph 就會被建立。因此一個 TensorFlow Function 其實是 Backed By 很多 TensorFlow Graph。

In [12]:
@tf.function
def my_relu(x):
    return tf.maximum(0., x)

In [13]:
# 3 個 TensorFlow Graph 被建立
print(my_relu(tf.constant(5.0)))
print(my_relu(tf.constant([1.0, -1.0, 3.0])))
print(my_relu(tf.constant([1.0, -1.0])))

tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor([1. 0. 3.], shape=(3,), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)


In [14]:
print(my_relu.pretty_printed_concrete_signatures())

my_relu(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()

my_relu(x)
  Args:
    x: float32 Tensor, shape=(2,)
  Returns:
    float32 Tensor, shape=(2,)

my_relu(x)
  Args:
    x: float32 Tensor, shape=(3,)
  Returns:
    float32 Tensor, shape=(3,)


#### TensorFlow Function: Graph Execution vs Eager Execution

照理說，相同的 Code 不管是 Graph Execution 還是 Eager Execution 必須產生相同的結果；然而，在某些情況下會不同。

In [15]:
@tf.function
def get_MSE(y_true, y_pred):
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)

In [16]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)

print(y_true)
print(y_pred)

tf.Tensor([6 6 3 1 9], shape=(5,), dtype=int32)
tf.Tensor([4 0 3 8 7], shape=(5,), dtype=int32)


In [17]:
get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=18>

In [18]:
# TensorFlow Function 預設是 Graph Execution，手動調成 Eager Execution
tf.config.run_functions_eagerly(True)

In [19]:
# 發現 Graph Execution 與 Eager Execution 結果相同
get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=18>

In [20]:
# 必須再手動調回來
tf.config.run_functions_eagerly(False)

在某些情況下，Graph Execution 與 Eager Execution 的結果會不同。例如，在 TensorFlow Function 中加入 print() statement

In [21]:
@tf.function
def get_MSE(y_true, y_pred):
    sq_diff = tf.pow(y_true - y_pred, 2)
    print("get_MSE is called !")
    return tf.reduce_mean(sq_diff)

In [22]:
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

get_MSE is called !


由上面程式碼發現 print() 只有被執行一次！實際上，當我們呼叫 TensorFlow Function 時，TensorFlow Function 中所有的 Code 會被 Trace 一次，決定哪些 Code 要用來建立 TensorFlow Graph。print() 就是在 Tracing 過程被執行的。

因為 print() 不會被放進去 TensorFlow Graph 中，因此當我們呼叫三次 get_MSE() 時，TensorFlow Graph 被執行了三次，卻沒有顯示任何東西。

如果將 TensorFlow Function 以 Eager Mode 執行，print() 就每一次都會被執行！

In [23]:
tf.config.run_functions_eagerly(True)

In [24]:
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

get_MSE is called !
get_MSE is called !
get_MSE is called !


In [25]:
tf.config.run_functions_eagerly(False)

#### Seeing the Spped Up

TensorFlow Function 能夠帶來多少 Speed Up 取決於 TensorFlow Function 中的 Operation。TensorFlow Function 常常用來加速 Training Loop。

TensorFlow Function 為了帶來 Speed Up 就必須付出代價 —— 建立 TensorFlow Graph 的 Overhead。因此，如果 TensorFlow Function 中的 Operation 很少，反而帶來的 Speed Up 都被「建立」Graph 的 Overhead 給吸收！

In [26]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
    result = tf.eye(10, dtype=tf.dtypes.int32)
    for _ in range(y):
        result = tf.matmul(x, result)
    return result

In [29]:
# eager execution
timeit.timeit(lambda: power(x, 100), number=1000)

1.0946794999999838

In [30]:
power = tf.function(power)

# graph execution
timeit.timeit(lambda: power(x, 100), number=1000)

0.21099879100006547

#### When is TensorFlow Function tracing

為了要知道 TensorFlow Function 在什麼時候 Tracing，最簡單即是在 TensorFlow Function 中放入 print()，因為只要 Tracing 發生，print() 就會被執行！

In [42]:
@tf.function
def tracing_function(x):
    print("Tracing")
    return x*x + tf.constant(2)

In [43]:
print("#1: ", tracing_function(tf.constant(5))) # tracing
print("#2: ", tracing_function(tf.constant(6))) # no tracing

Tracing
#1:  tf.Tensor(27, shape=(), dtype=int32)
#2:  tf.Tensor(38, shape=(), dtype=int32)


In [44]:
# 只要 "Input Signatur" 改變，就要重新 Tracing 建立 TensorFlow Graph
print("#1: ", tracing_function(5)) # tracing
print("#2: ", tracing_function(6)) # tracing

Tracing
#1:  tf.Tensor(27, shape=(), dtype=int32)
Tracing
#2:  tf.Tensor(38, shape=(), dtype=int32)
